In [1]:
import numpy as np
import pandas as pd
import os
from os import listdir
from os.path import isfile, join
import math

In [2]:
#Get relative path of the CSV File 
#Enumerate therough all the files in the folder and check if the path exists

mypath = os.path.dirname(os.path.realpath('train.csv'))
onlyfiles = [f for f in listdir(mypath) if isfile(join(mypath, f))]
os.path.exists(mypath)

True

In [3]:
print(onlyfiles[7])

train.csv


In [4]:
#Read Training CSV for house regression
dataset = pd.read_csv(onlyfiles[7])
dataset.shape
dataset.drop(['Id'], axis=1)
dataset.head()

,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,...,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice
0,1,60,RL,65.0,8450,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,2,2008,WD,Normal,208500
1,2,20,RL,80.0,9600,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,5,2007,WD,Normal,181500
2,3,60,RL,68.0,11250,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,9,2008,WD,Normal,223500
3,4,70,RL,60.0,9550,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,2,2006,WD,Abnorml,140000
4,5,60,RL,84.0,14260,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,12,2008,WD,Normal,250000


In [5]:
testDataset = pd.read_csv(onlyfiles[6])
testDataset.head()
testDataset.drop(['Id'], axis=1)

,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,LotConfig,...,ScreenPorch,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition
0,20,RH,80.0,11622,Pave,NaN,Reg,Lvl,AllPub,Inside,...,120,0,NaN,MnPrv,NaN,0,6,2010,WD,Normal
1,20,RL,81.0,14267,Pave,NaN,IR1,Lvl,AllPub,Corner,...,0,0,NaN,NaN,Gar2,12500,6,2010,WD,Normal
2,60,RL,74.0,13830,Pave,NaN,IR1,Lvl,AllPub,Inside,...,0,0,NaN,MnPrv,NaN,0,3,2010,WD,Normal
3,60,RL,78.0,9978,Pave,NaN,IR1,Lvl,AllPub,Inside,...,0,0,NaN,NaN,NaN,0,6,2010,WD,Normal
4,120,RL,43.0,5005,Pave,NaN,IR1,HLS,AllPub,Inside,...,144,0,NaN,NaN,NaN,0,1,2010,WD,Normal
5,60,RL,75.0,10000,Pave,NaN,IR1,Lvl,AllPub,Corner,...,0,0,NaN,NaN,NaN,0,4,2010,WD,Normal
6,20,RL,NaN,7980,Pave,NaN,IR1,Lvl,AllPub,Inside,...,0,0,NaN,GdPrv,Shed,500,3,2010,WD,Normal
7,60,RL,63.0,8402,Pave,NaN,IR1,Lvl,AllPub,Inside,...,0,0,NaN,NaN,NaN,0,5,2010,WD,Normal
8,20,RL,85.0,10176,Pave,NaN,Reg,Lvl,AllPub,Inside,...,0,0,NaN,NaN,NaN,0,2,2010,WD,Normal
9,20,RL,70.0,8400,Pave,NaN,Reg,Lvl,AllPub,Corner,...,0,0,NaN,MnPrv,NaN,0,4,2010,WD,Normal


importing the dataset 
encode categorical data
Backward elimination

In [6]:
#Delete redundant features with over 50% NAN values E.G: porch sizes, pool area, alley, miscallenous features and its value 
unwantedFeaturesList = ["WoodDeckSF","OpenPorchSF","EnclosedPorch","3SsnPorch","ScreenPorch","PoolArea",
                       "PoolQC","Fence","Alley","MiscFeature","MiscVal"]
dataset.drop(unwantedFeaturesList,inplace=True,axis=1)
testDataset.drop(unwantedFeaturesList,inplace=True,axis=1)

In [7]:
# Check all the columns fetures whcih has a numerical value type
is_number = np.vectorize(lambda x: np.issubdtype(x, np.number))
is_number(dataset.dtypes)

array([ True,  True, False,  True,  True, False, False, False, False,
       False, False, False, False, False, False, False,  True,  True,
        True,  True, False, False, False, False, False,  True, False,
       False, False, False, False, False, False,  True, False,  True,
        True,  True, False, False, False, False,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True, False,  True,
       False,  True, False, False,  True, False,  True,  True, False,
       False, False,  True,  True, False, False,  True], dtype=bool)

In [8]:
#Separate and select dataframes by dataTypes 
df_NonNumericalType = dataset.select_dtypes(exclude=[np.number])
df_NumericalType = dataset.select_dtypes(include=[np.number])

#prepare test dataset
df_TestNonNumericalType = testDataset.select_dtypes(exclude=[np.number])
df_TestNumericalType = testDataset.select_dtypes(include=[np.number])

#Set type to categorical
df_NonNumericalType[df_NonNumericalType.select_dtypes(['object']).columns] = df_NonNumericalType.select_dtypes(['object']).apply(lambda x: x.astype('category'))
df_TestNonNumericalType[df_TestNonNumericalType.select_dtypes(['object']).columns] = df_TestNonNumericalType.select_dtypes(['object']).apply(lambda x: x.astype('category'))


/Users/MM/anaconda/lib/python3.6/site-packages/pandas/core/frame.py:2440: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self[k1] = value[k2]


In [9]:
df_NumericalType.drop(["Id"], inplace=True, axis=1)
df_NumericalType.drop(['SalePrice'], inplace=True, axis=1)
df_TestNumericalType.drop(["Id"], inplace=True, axis=1)

/Users/MM/anaconda/lib/python3.6/site-packages/ipykernel/__main__.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':
/Users/MM/anaconda/lib/python3.6/site-packages/ipykernel/__main__.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  from ipykernel import kernelapp as app
/Users/MM/anaconda/lib/python3.6/site-packages/ipykernel/__main__.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  app.launch_new_instance()


In [10]:
df_NumericalType.head()

,MSSubClass,LotFrontage,LotArea,OverallQual,OverallCond,YearBuilt,YearRemodAdd,MasVnrArea,BsmtFinSF1,BsmtFinSF2,...,HalfBath,BedroomAbvGr,KitchenAbvGr,TotRmsAbvGrd,Fireplaces,GarageYrBlt,GarageCars,GarageArea,MoSold,YrSold
0,60,65.0,8450,7,5,2003,2003,196.0,706,0,...,1,3,1,8,0,2003.0,2,548,2,2008
1,20,80.0,9600,6,8,1976,1976,0.0,978,0,...,0,3,1,6,1,1976.0,2,460,5,2007
2,60,68.0,11250,7,5,2001,2002,162.0,486,0,...,1,3,1,6,1,2001.0,2,608,9,2008
3,70,60.0,9550,7,5,1915,1970,0.0,216,0,...,0,3,1,7,1,1998.0,3,642,2,2006
4,60,84.0,14260,8,5,2000,2000,350.0,655,0,...,1,4,1,9,1,2000.0,3,836,12,2008


In [11]:
#df_NumericalType.drop(["Id"], inplace=True, axis=1)
#replace all the NA values to numpy NAN for float 64 data type
df_NumericalType.replace('NA',np.nan)
df_TestNumericalType.replace('NA',np.nan)
#Select Numerical values except the index 
xNumerical = df_NumericalType.values
xTestNumerical = df_TestNumericalType.values

In [12]:
df_NumericalType.shape, df_TestNumericalType.shape, len(df_NumericalType.T)

((1460, 29), (1459, 29), 29)

In [13]:
#Handle missing data for numerical values 
from sklearn.preprocessing import Imputer
if df_TestNumericalType.isnull().values.any() :
    #mean is the deafualt parameter anywyas
    imputer = Imputer(missing_values = 'NaN', strategy='mean', axis=0)
    imputer.fit(xTestNumerical[:,0:29])
    xTestNumerical[:,0:29] = imputer.transform(xTestNumerical[:,0:29])

In [14]:
pd.DataFrame(xTestNumerical).head()

,0,1,2,3,4,5,6,7,8,9,...,19,20,21,22,23,24,25,26,27,28
0,20.0,80.0,11622.0,5.0,6.0,1961.0,1961.0,0.0,468.0,144.0,...,0.0,2.0,1.0,5.0,0.0,1961.0,1.0,730.0,6.0,2010.0
1,20.0,81.0,14267.0,6.0,6.0,1958.0,1958.0,108.0,923.0,0.0,...,1.0,3.0,1.0,6.0,0.0,1958.0,1.0,312.0,6.0,2010.0
2,60.0,74.0,13830.0,5.0,5.0,1997.0,1998.0,0.0,791.0,0.0,...,1.0,3.0,1.0,6.0,1.0,1997.0,2.0,482.0,3.0,2010.0
3,60.0,78.0,9978.0,6.0,6.0,1998.0,1998.0,20.0,602.0,0.0,...,1.0,3.0,1.0,7.0,1.0,1998.0,2.0,470.0,6.0,2010.0
4,120.0,43.0,5005.0,8.0,5.0,1992.0,1992.0,0.0,263.0,0.0,...,0.0,2.0,1.0,5.0,0.0,1992.0,2.0,506.0,1.0,2010.0


In [15]:
#Handle missing data for numerical values 
from sklearn.preprocessing import Imputer
if df_NumericalType.isnull().values.any() :
    #mean is the deafualt parameter anywyas
    imputer = Imputer(missing_values = 'NaN', strategy='mean', axis=0)
    imputer.fit(xNumerical[:,0:28])
    xNumerical[:,0:28] = imputer.transform(xNumerical[:,0:28])
pd.DataFrame(xNumerical).head()

,0,1,2,3,4,5,6,7,8,9,...,19,20,21,22,23,24,25,26,27,28
0,60.0,65.0,8450.0,7.0,5.0,2003.0,2003.0,196.0,706.0,0.0,...,1.0,3.0,1.0,8.0,0.0,2003.0,2.0,548.0,2.0,2008.0
1,20.0,80.0,9600.0,6.0,8.0,1976.0,1976.0,0.0,978.0,0.0,...,0.0,3.0,1.0,6.0,1.0,1976.0,2.0,460.0,5.0,2007.0
2,60.0,68.0,11250.0,7.0,5.0,2001.0,2002.0,162.0,486.0,0.0,...,1.0,3.0,1.0,6.0,1.0,2001.0,2.0,608.0,9.0,2008.0
3,70.0,60.0,9550.0,7.0,5.0,1915.0,1970.0,0.0,216.0,0.0,...,0.0,3.0,1.0,7.0,1.0,1998.0,3.0,642.0,2.0,2006.0
4,60.0,84.0,14260.0,8.0,5.0,2000.0,2000.0,350.0,655.0,0.0,...,1.0,4.0,1.0,9.0,1.0,2000.0,3.0,836.0,12.0,2008.0


In [16]:
np.isnan(xTestNumerical).any()

False

In [17]:
#Scaling Numerical data
from sklearn.preprocessing import StandardScaler
sc_X= StandardScaler()
NumericalScaledData = sc_X.fit_transform(xNumerical)
TestNumericalScaledData = sc_X.fit_transform(xTestNumerical)

In [18]:
df_AllNonNumerical = pd.concat([df_NonNumericalType, df_TestNonNumericalType], axis=0)
df_AllNonNumerical[df_AllNonNumerical.select_dtypes(['object']).columns] = df_AllNonNumerical.select_dtypes(['object']).apply(lambda x: x.astype('category'))
cat_features = [key for key in dict(df_AllNonNumerical.dtypes) if dict(df_AllNonNumerical.dtypes)[key] in ['category']]
df_AllNonNumericalEncoded = pd.get_dummies(df_AllNonNumerical, columns=cat_features, drop_first=True)
df_NonNumericalencoded = df_AllNonNumericalEncoded[:len(df_NonNumericalType.values)]

#Encode Test numerical data
df_TestNonNumericalencoded = df_AllNonNumericalEncoded[len(df_NonNumericalType.values):]


In [19]:
df_AllNonNumericalEncoded.shape,df_NonNumericalencoded.shape, df_TestNonNumericalencoded.shape

((2919, 200), (1460, 200), (1459, 200))

In [20]:
#Sort by categorical data and encoding it
#Create dummy data for all the categorical columns
#cat_features = [key for key in dict(df_NonNumericalType.dtypes) if dict(df_NonNumericalType.dtypes)[key] in ['category']]
#df_NonNumericalencoded = pd.get_dummies(df_NonNumericalType, columns=cat_features, drop_first=True)
#cat_features = [key for key in dict(df_TestNonNumericalType.dtypes) if dict(df_TestNonNumericalType.dtypes)[key] in ['category']]
#df_TestNonNumericalencoded = pd.get_dummies(df_TestNonNumericalType, columns=cat_features, drop_first=True)
#df_NonNumericalencoded.reset_index(level=0, inplace=True)
#df_NumericalType.reset_index(level=0, inplace=True)

In [21]:
df_NumericalScaled = pd.DataFrame(NumericalScaledData)
df_NumericalScaled.columns = df_NumericalType.columns
df_AllTypes = pd.concat([df_NumericalScaled, df_NonNumericalencoded], axis=1)
df_AllTypes.shape

(1460, 229)

-Prepare test data for future testing purpose.
-16 features less due to categorical dummy varaibles introduction in previous steps

In [22]:
df_TestNumericalScaled = pd.DataFrame(TestNumericalScaledData)
df_TestNumericalScaled.columns = df_TestNumericalType.columns
df_AllTestTypes = pd.concat([df_TestNumericalScaled, df_TestNonNumericalencoded], axis=1)
df_AllTestTypes.shape

(1459, 229)

In [23]:
df_TestNumericalScaled.shape, df_TestNonNumericalencoded.shape

((1459, 29), (1459, 200))

In [24]:
# instantiate the independent variable
from sklearn.tree import *
Y = dataset.iloc[:,-1]

In [25]:
from sklearn.feature_selection import VarianceThreshold
sel = VarianceThreshold(threshold=(.5 * (1 - .5)))
sel2 = VarianceThreshold(threshold=(.5 * (1 - .5)))
varianceThresholdTrain = sel.fit_transform(df_AllTypes.values)
varianceThresholdTest = sel2.fit_transform(df_AllTestTypes.values)

In [88]:
from sklearn.svm import LinearSVC
from sklearn.feature_selection import SelectFromModel
lsvc = LinearSVC(C=0.04, penalty="l1", dual=False).fit(df_AllTypes, Y.values)
model = SelectFromModel(lsvc, prefit=True)
X_new = model.transform(df_AllTypes)
X_new.shape

(1460, 47)

dtype('int64')

In [95]:
#Split training dataset into test and train dataset
import matplotlib.pyplot as plt
from sklearn.cross_validation import train_test_split
X_train,X_test,Y_train,Y_test = train_test_split(X_new,Y.values, test_size = 0.3)

In [96]:
#Using Decesion tree regressor to score the split test data
reg = DecisionTreeRegressor(min_samples_leaf=5, max_depth=3,random_state = 0)
reg.fit(X_train, Y_train)
reg.score(X_test, Y_test)

0.68841580250640289

In [97]:
#Using Random forest regressor to score
from sklearn.ensemble import RandomForestRegressor
reg2 = RandomForestRegressor(n_estimators = 500, random_state = 0,oob_score=True)
reg2.fit(X_train,Y_train)
reg2.score(X_test,Y_test),reg2.oob_score_

(0.82245844330321116, 0.82794233820283369)

In [98]:
Y1_pred = reg.predict(X_test)
Y2_pred = reg2.predict(X_test)

Root mean square error log value for descision tree

In [99]:
cv_rmsel=0
rmsel=0
for i in range(len(Y_test)):
    rmsel += (math.log(Y_test[i]+1) - math.log(Y1_pred[i]+1))**2
rmsel=(rmsel/len(Y_test))**0.5
print(rmsel)

0.21068285908987402


Root mean square error log value for random forest

In [100]:
cv_rmsel=0
rmsel=0
for i in range(len(Y_test)):
    rmsel += (math.log(Y_test[i]+1) - math.log(Y2_pred[i]+1))**2
rmsel=(rmsel/len(Y_test))**0.5
print(rmsel)

0.13916702756556903


In [ ]:
kagReg = RandomForestRegressor(n_estimators = 100, random_state = 0,oob_score=True)
kagReg.fit(varianceThresholdTrain, Y)
predictionVal = kagReg.predict(varianceThresholdTrain)

In [ ]:
df_AllTypes.shape, df_AllTestTypes.shape, Y.shape, varianceThresholdTrain.shape,varianceThresholdTrain.shape

In [ ]:
prediction = pd.DataFrame()
prediction = pd.concat([testDataset['Id'],pd.DataFrame(predictionVal)],axis=1)
prediction.columns=["Id","SalePrice"]
#prediction.to_csv('submission2.csv',index=False)